In [ ]:
import numpy as np
import pandas as pd
import joblib

# load data
df_new = pd.read_csv(r"c:\Users\Lenovo\Desktop\MOF_H2\MOF_catalysis\Data\Prediction\Core_MOF.csv",encoding = "latin1") 
df_new = df_new.dropna()
df_new = df_new[~((df_new['L_MolWt'] == 0) | (df_new['L_MolLogP'] == 0) | (df_new['L_TPSA'] == 0))]
# load model
selected_features = joblib.load(r'c:\Users\Lenovo\Desktop\MOF_H2\MOF_catalysis\Model\rfecv_feature_selector.pkl')
scaler_full = joblib.load(r'c:\Users\Lenovo\Desktop\MOF_H2\MOF_catalysis\Model\scaler.pkl')   
lightgbm_model = joblib.load(r'c:\Users\Lenovo\Desktop\MOF_H2\MOF_catalysis\Model\lightgbm_best_model.pkl')  

# select the same features
X_new = df_new[selected_features]
X_new_selected = X_new[selected_features]
X_new_scaled = scaler_full.transform(X_new_selected)

# predict
y_pred_lightgbm = lightgbm_model.predict(X_new_scaled)
y_pred_proba_lightgbm = lightgbm_model.predict_proba(X_new_scaled)[:, 1]


df_new['LightGBM_Prediction'] = y_pred_lightgbm
df_new['LightGBM_Probability'] = y_pred_proba_lightgbm

# Count LightGBM-predicted low-performance (label = 0) and high-performance (label = 1) 
lightgbm_counts = df_new['LightGBM_Prediction'].value_counts().rename({0: 'Low', 1: 'High'})

print("\n=== LightGBM Stability Count ===")
print(lightgbm_counts)



=== LightGBM Stability Count ===
Low     7872
High    1736
Name: LightGBM_Prediction, dtype: int64
